In [1]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [2]:
ROI_data = pd.read_csv('ROI_data.csv')
ROI_data.columns[1:]

Index(['Print', 'TV', 'SEO', 'AdWords', 'Facebook', 'LinkedIn', 'Instagram',
       'Snapchat', 'Twitter', 'Email'],
      dtype='object')

In [3]:
ROI_values = ROI_data.iloc[0,:].to_list()[1:]

In [4]:
#Define optimization function 

def optimize(obj_func,constr_matrix,constr_signs, constr_limits, max_min):
    obj_mod = gp.Model()
    num_rows,num_cols = constr_matrix.shape
    obj_mod_X = obj_mod.addMVar(num_cols)
    obj_mod_cons = obj_mod.addMConstr(constr_matrix,obj_mod_X,constr_signs,constr_limits)
        
    if  max_min == 'max' :
        obj_mod.setMObjective(None, obj_func, 0, sense = gp.GRB.MAXIMIZE)
    else :
        obj_mod.setMObjective(None, obj_func, 0, sense = gp.GRB.MINIMIZE)
    
    obj_mod.Params.OutputFlag = 0
    obj_mod.Params.TimeLimit = 3600

    obj_mod.optimize()
    
    optimal_val = obj_mod.objVal 
    variable_val_at_optimal_soln = obj_mod_X.x

    return optimal_val, variable_val_at_optimal_soln

## Task 3

In [5]:
obj_func = np.array(ROI_values) #define objective vector
constr_matrix = np.zeros((13,10)) #initializing constraint matrix
constr_matrix[0,:] = [1,1,1,1,1,1,1,1,1,1]
constr_matrix[1,:] = [-1,-1,0,0,1,0,0,0,0,1] 
constr_matrix[2,:] = [0,0,-2,-2,1,1,1,1,1,0]
constr_matrix[3:,:] = np.identity(10)
constr_limits = [10,0,0,3,3,3,3,3,3,3,3,3,3]
constr_signs = ['<','>','>','<','<','<','<','<','<','<','<','<','<']

In [6]:
optimal_value_1, variable_values= optimize(obj_func,constr_matrix,constr_signs,constr_limits,'max')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14


In [203]:
optimal_value_1

0.45600000000000007

In [204]:
variable_values

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

## Task 4

In [7]:
ROI_values_second_firm = ROI_data.iloc[1,:].to_list()[1:]

In [8]:
obj_func_2 = np.array(ROI_values_second_firm)

In [9]:
optimize(obj_func_2,constr_matrix,constr_signs,constr_limits,'max')

(0.45600000000000007, array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.]))

## Task 5

In [203]:
optimal_val

0.45600000000000007

In [204]:
variable_val_at_optimal_soln

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

## Task 6

In [10]:
obj_mod = gp.Model()
num_rows,num_cols = constr_matrix.shape
obj_mod_X = obj_mod.addMVar(num_cols)
obj_mod_cons = obj_mod.addMConstr(constr_matrix,obj_mod_X,constr_signs,constr_limits)

obj_mod.setMObjective(None, obj_func, 0, sense = gp.GRB.MAXIMIZE)
obj_mod.Params.OutputFlag = 0
obj_mod.Params.TimeLimit = 3600

obj_mod.optimize()

optimal_val = obj_mod.objVal 
variable_val_at_optimal_soln = obj_mod_X.x

In [11]:
optimal_val

0.45600000000000007

In [12]:
variable_val_at_optimal_soln

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

In [13]:
low = obj_mod_X.SAObjLow

In [14]:
high = obj_mod_X.SAObjUp

In [15]:
list1 = [low,high]

In [16]:
low_high_limits = pd.DataFrame(list1).rename(index = {0 : 'Low', 1: 'High'})

In [17]:
low_high_limits.columns = ROI_data.columns[1:]

In [18]:
low_high_limits

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Low,-inf,0.039,-inf,0.033,-inf,-inf,0.039,-inf,-inf,0.029
High,0.049,0.062,0.039,0.046,0.029,0.039,inf,0.039,0.039,inf


## Task 7

In [25]:
roi_monthly_data = pd.read_csv('roi_mat.csv')

In [26]:
roi_monthly_data.set_index(roi_monthly_data.columns[0], inplace = True)

In [27]:
roi_monthly_data_array = np.array(roi_monthly_data)

In [140]:
constr_limits = [10,0,0,3,3,3,3,3,3,3,3,3,3]

In [141]:
roi_monthly_data_array
constr_limits

[10, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [142]:
optimal_obj_val = []
optimal_variable_vals = np.zeros((12,10))
monthly_budget = []
for i in range(len(roi_monthly_data_array)):
    print("budget for month ",i+1,":",constr_limits[0])
    monthly_budget.append(constr_limits[0])
    obj_funct_i = roi_monthly_data_array[i]
    #print(obj_funct_i)
    #optimal_obj_val.append(optimize(obj_funct_i,constr_matrix,constr_signs, constr_limits, 'max')[0])
    roi_1 = optimize(obj_funct_i,constr_matrix,constr_signs, constr_limits, 'max')[0]
    optimal_obj_val.append(roi_1)
    optimal_variable_vals[i,:] = (optimize(obj_funct_i,constr_matrix,constr_signs, constr_limits, 'max')[1])
    print("ROI for month",i+1,':', roi_1)
    constr_limits[0] = constr_limits[0] + (roi_1*0.01)/2
#     constr_limits[0] = (10 + (optimal_obj_val[i]/2))

budget for month  1 : 10
ROI for month 1 : 37.3
budget for month  2 : 10.1865
ROI for month 2 : 40.629599999999996
budget for month  3 : 10.389648000000001
ROI for month 3 : 41.4416976
budget for month  4 : 10.596856488
ROI for month 4 : 41.448683356800004
budget for month  5 : 10.804099904784
ROI for month 5 : 43.214349666744
budget for month  6 : 11.02017165311772
ROI for month 6 : 45.47665228184733
budget for month  7 : 11.247554914526956
ROI for month 7 : 46.86546416665513
budget for month  8 : 11.481882235360231
ROI for month 8 : 48.79661131005956
budget for month  9 : 11.725865291910528
ROI for month 9 : 45.92199484466448
budget for month  10 : 11.95547526613385
ROI for month 10 : 42.75752085162832
budget for month  11 : 12.169262870391993
ROI for month 11 : 51.73755681180984
budget for month  12 : 12.427950654451042
ROI for month 12 : 51.68341742146885


In [143]:
optimal_obj_val
monthly_allocation = pd.DataFrame(optimal_variable_vals).rename(index = {0 :'January', 1:'February', 2: 'March', 3:'April', 4:'May', 5:'June', 6:'July', 7:'August', 8:'September', 9:'October', 10:'November', 11: 'December'})
monthly_allocation.columns = ROI_data.columns[1:]
monthly_allocation

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.000000
February,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.000000
March,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.389648,0.0,3.000000,0.000000
April,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.596856,0.000000
May,1.804100,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.000000
June,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.020172,3.000000
July,1.123777,0.0,0.0,3.000000,1.123777,0.000000,3.000000,0.0,3.000000,0.000000
August,3.000000,0.0,0.0,1.827294,0.000000,0.654588,0.000000,0.0,3.000000,3.000000
September,1.362933,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.362933
October,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,0.000000,2.955475


In [147]:
monthly_allocation['Monthly_budget'] = monthly_budget
monthly_allocation['Max_ROI'] = optimal_obj_val
monthly_allocation

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Max_ROI,Monthly_budget
January,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.000000,37.300000,10.000000
February,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.000000,40.629600,10.186500
March,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.389648,0.0,3.000000,0.000000,41.441698,10.389648
April,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.596856,0.000000,41.448683,10.596856
May,1.804100,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.000000,43.214350,10.804100
June,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.020172,3.000000,45.476652,11.020172
July,1.123777,0.0,0.0,3.000000,1.123777,0.000000,3.000000,0.0,3.000000,0.000000,46.865464,11.247555
August,3.000000,0.0,0.0,1.827294,0.000000,0.654588,0.000000,0.0,3.000000,3.000000,48.796611,11.481882
September,1.362933,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.362933,45.921995,11.725865
October,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,0.000000,2.955475,42.757521,11.955475


In [190]:
change = np.zeros((12,10))
monthly_allocation_array = np.array(monthly_allocation)
for month in range(0, len(monthly_allocation_array)-1):
    for platform in range(len(monthly_allocation_array)-2):
        change[month,platform] = (monthly_allocation_array[month+1,platform] - monthly_allocation_array[month,platform])
        

In [202]:
change_per_platform = pd.DataFrame(change).rename(index = {0 :'January', 1:'February', 2: 'March', 3:'April', 4:'May', 5:'June', 6:'July', 7:'August', 8:'September', 9:'October', 10:'November', 11: 'December'})
change_per_platform.columns = ROI_data.columns[1:]
change_per_platform.shift(periods = 1)

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
February,0.000000,0.0,0.0,1.062167,3.000000,0.000000,-2.666667,0.0,1.791000,-3.000000
March,-3.000000,0.0,0.0,0.604500,-3.000000,3.000000,1.389648,0.0,1.209000,0.000000
April,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.610352,0.0,-1.403144,0.000000
May,1.804100,0.0,0.0,-3.000000,0.000000,-3.000000,0.000000,0.0,1.403144,3.000000
June,1.195900,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-0.979828,0.000000
July,-1.876223,0.0,0.0,3.000000,1.123777,0.000000,0.000000,0.0,0.979828,-3.000000
August,1.876223,0.0,0.0,-1.172706,-1.123777,0.654588,-3.000000,0.0,0.000000,3.000000
September,-1.637067,0.0,0.0,1.172706,0.000000,2.345412,0.000000,0.0,0.000000,-1.637067
October,-1.362933,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,-3.000000,1.592543


In [ ]:
It is not stable budget 